# SQL i kamień filozoficzny
## SQLAlchemy

### Marcin Jaroszewski
### 05.IV.2018, Python Level UP

![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/logo.png)

![Plan zajęć](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/plan_zajec.png)

# 1. ORM - Object-Relational Mapping

Sklejenie dwóch niezgodnych systemów typów za pomocą programowania obiektowego.


Opis na wikipedii: https://en.wikipedia.org/wiki/Object-relational_mapping

# 2. ORM w Python 

- SQLALchemy: https://www.sqlalchemy.org/
- PonyORM: https://ponyorm.com/
- Django ORM: https://www.djangoproject.com/

# 3. SQLAlchemy

- Duże.
- Rozwijane przez wiele lat.
- Powszechnie używane.
- Nie zawsze łatwe w obsłudze.

Będziemy się zajmować częścią zwaną **ORM**.

# 4. Instalacja

Będziemy używać SQLAlchemy razem z Flask więc proponuję zainstalować je w tym samym środowisku wirtualnym.

```bash
pip install SQLAlchemy
```

instrukcja: http://docs.sqlalchemy.org/en/latest/intro.html#install-via-pip

# 5. Modele i relacje

Mamy dwa przypadki pracy z SQLAlchemy:
1. Nowy projekt, bazy jeszcze nie ma - od początku wdrażamy SQLAlchemy.  
W takim wypadku możemy schemat bazy danych wygenerować z modeli SQLAchemy.

2. Już istniejący projekt - SQLAlchemy podłączamy do już istniejącej bazy danych.  
Modele (niepełne) możemy wygenerować "automatycznie" lub ręcznie (pełne).

W obu przypadkach lepiej dla nas i naszej aplikacji, aby modele były zgodne ze schematem bazy. W przeciwnym wypadku złe rzeczy mogą się stać.

500 to łagodny wymiar kary.

SQLAlchemy obiecuje nam dostęp do bazy danych i danych w niej zawartych w obiektowy i puyhonowy sposób. Zweryfikujemy te obietnice w praktyce łacząc się z bazą Sakila. 

Pierwszy krok w wykorzystaniu SQLAlchemy wymaga zamodelowania bazy danych (schema).

Chcemy się połaczyć do istniejącej bazy wypełnionej danymi.

Możemy do problemu podejść na dwa sposoby:
1. Ręczny - sami napiszemy modele pasujące do istniejącej bazy.
2. Automatyczny - coś wygeneruje modele za nas.

Ja należę do ludzi leniwych i chciałbym, żeby komputer pracował za mnie więc zacznę od podejścia automatycznego.

Dokumentacja: http://docs.sqlalchemy.org/en/latest/orm/extensions/automap.html
Uwaga: http://docs.sqlalchemy.org/en/latest/core/reflection.html

In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

Base = automap_base()

engine = create_engine("sqlite:///sakila.db")
Base.prepare(engine, reflect=True)
print(Base.classes.actor)

<class 'sqlalchemy.ext.automap.actor'>


In [2]:
print(repr(Base.classes.actor.__table__))

Table('actor', MetaData(bind=None), Column('actor_id', INTEGER(), table=<actor>, primary_key=True, nullable=False), Column('first_name', VARCHAR(length=45), table=<actor>, nullable=False), Column('last_name', VARCHAR(length=45), table=<actor>, nullable=False), Column('last_update', TIMESTAMP(), table=<actor>), schema=None)


Fajnie, pięknie, automat zadziałał.

Ale my liczyliśmy na pełen kod modeli :(.

Coś w styllu: https://pypi.python.org/pypi/sqlacodegen mogłoby nam pomóc.
Jednak ten projekt ma ponad 2 lata, a jego repozytorium zostało skasowane :(.

Do odważnych, świat należy.

Spróbowałem `sqlacodegen` i "zadziałało". Coś zostało wygenerowane i jest dostępne w `auto_generated_sakila_models.py`. Zanim użyjemy automatycznie wygenerowanych modeli wypadałoby je sprawdzić (ręcznie?).

Interesują nas głównie modele `Actor` i `Film` wraz z powiązaniami.

In [3]:
# coding: utf-8
from sqlalchemy import CheckConstraint, Column, DateTime, ForeignKey, Index, Integer, LargeBinary, Numeric, SmallInteger, String, Table, Text, text
from sqlalchemy.orm import relationship
from sqlalchemy.sql.sqltypes import NullType
from sqlalchemy.ext.declarative import declarative_base


Base = declarative_base()
metadata = Base.metadata


class Actor(Base):
    __tablename__ = 'actor'

    actor_id = Column(Integer, primary_key=True)
    first_name = Column(String(45), nullable=False)
    last_name = Column(String(45), nullable=False)
    last_update = Column(DateTime)


class Addres(Base):
    __tablename__ = 'address'

    address_id = Column(Integer, primary_key=True)
    address = Column(String(50), nullable=False)
    address2 = Column(String(50), server_default=text("NULL"))
    district = Column(String(20), nullable=False)
    city_id = Column(ForeignKey('city.city_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    postal_code = Column(String(10), server_default=text("NULL"))
    phone = Column(String(20), nullable=False)
    last_update = Column(DateTime, nullable=False)

    city = relationship('City')


class Category(Base):
    __tablename__ = 'category'

    category_id = Column(SmallInteger, primary_key=True)
    name = Column(String(25), nullable=False)
    last_update = Column(DateTime, nullable=False)


class City(Base):
    __tablename__ = 'city'

    city_id = Column(Integer, primary_key=True)
    city = Column(String(50), nullable=False)
    country_id = Column(ForeignKey('country.country_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    last_update = Column(DateTime, nullable=False)

    country = relationship('Country')

    adresses = relationship('Addres')


class Country(Base):
    __tablename__ = 'country'

    country_id = Column(SmallInteger, primary_key=True)
    country = Column(String(50), nullable=False)
    last_update = Column(DateTime)


class Customer(Base):
    __tablename__ = 'customer'

    customer_id = Column(Integer, primary_key=True)
    store_id = Column(ForeignKey('store.store_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    first_name = Column(String(45), nullable=False)
    last_name = Column(String(45), nullable=False, index=True)
    email = Column(String(50), server_default=text("NULL"))
    address_id = Column(ForeignKey('address.address_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    active = Column(String(1), nullable=False, server_default=text("'Y'"))
    create_date = Column(DateTime, nullable=False)
    last_update = Column(DateTime, nullable=False)

    address = relationship('Addres')
    store = relationship('Store')


t_customer_list = Table(
    'customer_list', metadata,
    Column('ID', Integer),
    Column('name', NullType),
    Column('address', String(50)),
    Column('zip_code', String(10)),
    Column('phone', String(20)),
    Column('city', String(50)),
    Column('country', String(50)),
    Column('notes', NullType),
    Column('SID', Integer)
)


class Film(Base):
    __tablename__ = 'film'
    __table_args__ = (
        CheckConstraint("rating in ('G','PG','PG-13','R','NC-17')"),
    )

    film_id = Column(Integer, primary_key=True)
    title = Column(String(255), nullable=False)
    description = Column(Text, server_default=text("NULL"))
    release_year = Column(String(4), server_default=text("NULL"))
    language_id = Column(ForeignKey('language.language_id'), nullable=False, index=True)
    original_language_id = Column(ForeignKey('language.language_id'), index=True, server_default=text("NULL"))
    rental_duration = Column(SmallInteger, nullable=False, server_default=text("3"))
    rental_rate = Column(Numeric(4, 2), nullable=False, server_default=text("4.99"))
    length = Column(SmallInteger, server_default=text("NULL"))
    replacement_cost = Column(Numeric(5, 2), nullable=False, server_default=text("19.99"))
    rating = Column(String(10), server_default=text("'G'"))
    special_features = Column(String(100), server_default=text("NULL"))
    last_update = Column(DateTime, nullable=False)

    language = relationship('Language', primaryjoin='Film.language_id == Language.language_id')
    original_language = relationship('Language', primaryjoin='Film.original_language_id == Language.language_id')


class FilmActor(Base):
    __tablename__ = 'film_actor'

    actor_id = Column(ForeignKey('actor.actor_id', ondelete='NO ACTION', onupdate='CASCADE'), primary_key=True, nullable=False, index=True)
    film_id = Column(ForeignKey('film.film_id', ondelete='NO ACTION', onupdate='CASCADE'), primary_key=True, nullable=False, index=True)
    last_update = Column(DateTime, nullable=False)

    actor = relationship('Actor')
    film = relationship('Film')


class FilmCategory(Base):
    __tablename__ = 'film_category'

    film_id = Column(ForeignKey('film.film_id', ondelete='NO ACTION', onupdate='CASCADE'), primary_key=True, nullable=False, index=True)
    category_id = Column(ForeignKey('category.category_id', ondelete='NO ACTION', onupdate='CASCADE'), primary_key=True, nullable=False, index=True)
    last_update = Column(DateTime, nullable=False)

    category = relationship('Category')
    film = relationship('Film')


t_film_list = Table(
    'film_list', metadata,
    Column('FID', Integer),
    Column('title', String(255)),
    Column('description', Text),
    Column('category', String(25)),
    Column('price', Numeric(4, 2)),
    Column('length', SmallInteger),
    Column('rating', String(10)),
    Column('actors', NullType)
)


class FilmText(Base):
    __tablename__ = 'film_text'

    film_id = Column(SmallInteger, primary_key=True)
    title = Column(String(255), nullable=False)
    description = Column(Text)


class Inventory(Base):
    __tablename__ = 'inventory'
    __table_args__ = (
        Index('idx_fk_film_id_store_id', 'store_id', 'film_id'),
    )

    inventory_id = Column(Integer, primary_key=True)
    film_id = Column(ForeignKey('film.film_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    store_id = Column(ForeignKey('store.store_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False)
    last_update = Column(DateTime, nullable=False)

    film = relationship('Film')
    store = relationship('Store')


class Language(Base):
    __tablename__ = 'language'

    language_id = Column(SmallInteger, primary_key=True)
    name = Column(String(20), nullable=False)
    last_update = Column(DateTime, nullable=False)


class Payment(Base):
    __tablename__ = 'payment'

    payment_id = Column(Integer, primary_key=True)
    customer_id = Column(ForeignKey('customer.customer_id'), nullable=False, index=True)
    staff_id = Column(ForeignKey('staff.staff_id'), nullable=False, index=True)
    rental_id = Column(ForeignKey('rental.rental_id', ondelete='SET NULL', onupdate='CASCADE'), server_default=text("NULL"))
    amount = Column(Numeric(5, 2), nullable=False)
    payment_date = Column(DateTime, nullable=False)
    last_update = Column(DateTime, nullable=False)

    customer = relationship('Customer')
    rental = relationship('Rental')
    staff = relationship('Staff')


class Rental(Base):
    __tablename__ = 'rental'
    __table_args__ = (
        Index('idx_rental_uq', 'rental_date', 'inventory_id', 'customer_id', unique=True),
    )

    rental_id = Column(Integer, primary_key=True)
    rental_date = Column(DateTime, nullable=False)
    inventory_id = Column(ForeignKey('inventory.inventory_id'), nullable=False, index=True)
    customer_id = Column(ForeignKey('customer.customer_id'), nullable=False, index=True)
    return_date = Column(DateTime, server_default=text("NULL"))
    staff_id = Column(ForeignKey('staff.staff_id'), nullable=False, index=True)
    last_update = Column(DateTime, nullable=False)

    customer = relationship('Customer')
    inventory = relationship('Inventory')
    staff = relationship('Staff')


t_sales_by_film_category = Table(
    'sales_by_film_category', metadata,
    Column('category', String(25)),
    Column('total_sales', NullType)
)


t_sales_by_store = Table(
    'sales_by_store', metadata,
    Column('store_id', Integer),
    Column('store', NullType),
    Column('manager', NullType),
    Column('total_sales', NullType)
)


t_sqlite_sequence = Table(
    'sqlite_sequence', metadata,
    Column('name', NullType),
    Column('seq', NullType)
)


class Staff(Base):
    __tablename__ = 'staff'

    staff_id = Column(SmallInteger, primary_key=True)
    first_name = Column(String(45), nullable=False)
    last_name = Column(String(45), nullable=False)
    address_id = Column(ForeignKey('address.address_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    picture = Column(LargeBinary, server_default=text("NULL"))
    email = Column(String(50), server_default=text("NULL"))
    store_id = Column(ForeignKey('store.store_id', ondelete='NO ACTION', onupdate='CASCADE'), nullable=False, index=True)
    active = Column(SmallInteger, nullable=False, server_default=text("1"))
    username = Column(String(16), nullable=False)
    password = Column(String(40), server_default=text("NULL"))
    last_update = Column(DateTime, nullable=False)

    address = relationship('Addres')
    store = relationship('Store', primaryjoin='Staff.store_id == Store.store_id')


t_staff_list = Table(
    'staff_list', metadata,
    Column('ID', SmallInteger),
    Column('name', NullType),
    Column('address', String(50)),
    Column('zip_code', String(10)),
    Column('phone', String(20)),
    Column('city', String(50)),
    Column('country', String(50)),
    Column('SID', Integer)
)


class Store(Base):
    __tablename__ = 'store'

    store_id = Column(Integer, primary_key=True)
    manager_staff_id = Column(ForeignKey('staff.staff_id'), nullable=False, index=True)
    address_id = Column(ForeignKey('address.address_id'), nullable=False, index=True)
    last_update = Column(DateTime, nullable=False)

    address = relationship('Addres')
    manager_staff = relationship('Staff', primaryjoin='Store.manager_staff_id == Staff.staff_id')


## Relacje

* `relationship`: http://docs.sqlalchemy.org/en/latest/orm/relationships.html
* `backref`: http://docs.sqlalchemy.org/en/latest/orm/backref.html
* Jak się dostać do obiektów powiązanych.
* Czy odwołania do obiektów powiązanych kosztują?

## jeden-do-jeden

http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#one-to-one

## jeden-do-wielu

http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#one-to-many

## wiele-do-wielu

http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#many-to-many

Pokazać relację `Country` - `City`

# 6. Querowanie

Czyli pobieranie danych z bazy.

Musimy mieć jakieś "połączenie" do bazy danych.

Potrzebujemuy sesji: http://docs.sqlalchemy.org/en/latest/orm/session.html

In [4]:
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///sakila.db')
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
# Pobranie wszystkich aktorów z bazy
all_actors = session.query(Actor)

In [6]:
print(all_actors)

SELECT actor.actor_id AS actor_actor_id, actor.first_name AS actor_first_name, actor.last_name AS actor_last_name, actor.last_update AS actor_last_update 
FROM actor


In [7]:
print(type(all_actors))

<class 'sqlalchemy.orm.query.Query'>


Coś "nie pykło".

* Czym jest `Query`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query

In [8]:
# eksperymentujemy dalej
all_penelopes = all_actors.filter(Actor.first_name == 'PENELOPE')
print(id(all_actors))
print(id(all_penelopes))
print(all_penelopes)

139872189543704
139871952354608
SELECT actor.actor_id AS actor_actor_id, actor.first_name AS actor_first_name, actor.last_name AS actor_last_name, actor.last_update AS actor_last_update 
FROM actor 
WHERE actor.first_name = ?


In [9]:
for penelope in all_penelopes:
    print('{} {}'.format(penelope.first_name, penelope.last_name))

PENELOPE GUINESS
PENELOPE PINKETT
PENELOPE CRONYN
PENELOPE MONROE


In [10]:
print(all_penelopes.all())

[<__main__.Actor object at 0x7f367a08bd68>, <__main__.Actor object at 0x7f367a08bf98>, <__main__.Actor object at 0x7f367a08b908>, <__main__.Actor object at 0x7f367a09cba8>]


# 7. Wkładanie danych do bazy

In [11]:
marcin = Actor(
    first_name='Marcin',
    last_name='Jaroszewski'
)

In [12]:
marcin_q = session.query(Actor).filter(Actor.last_name == 'Jaroszewski').first()
print(marcin_q)

Ja chcę robić karierę na srebnym ekranie!

Dlaczego nie ma mnie w bazie aktorów?

In [13]:
marcin = Actor(
    first_name='Marcin',
    last_name='Jaroszewski'
)
session.add(marcin)
session.commit()
print(marcin.actor_id, marcin.first_name, marcin.last_name, marcin.last_update)

207 Marcin Jaroszewski None


In [14]:
marcin_q = session.query(Actor).filter(Actor.last_name == 'Jaroszewski').first()
print(marcin_q)
print(marcin_q.actor_id, marcin_q.first_name, marcin_q.last_name, marcin_q.last_update)

206 MARCIN Jaroszewski None


# 8. Zmienianie danych w bazie

In [15]:
marcin_q.first_name = 'MARCIN'
session.commit()

In [16]:
marcin_q = session.query(Actor).filter(Actor.last_name == 'Jaroszewski').first()
print(marcin_q)
print(marcin_q.actor_id, marcin_q.first_name, marcin_q.last_name, marcin_q.last_update)

206 MARCIN Jaroszewski None


# Pytania

## Co sprawia, że zapytanie zostanie wykonane?

* `all()`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.all
* `first()`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.first
* `one()`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.one
* `one_or_none`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.one_or_none
* `scalar`: http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.scalar
* iterowanie po wynikach

## Czym jest transakcja?

http://docs.sqlalchemy.org/en/latest/core/connections.html#sqlalchemy.engine.Transaction

## Co robi flush?

http://docs.sqlalchemy.org/en/latest/orm/session_api.html#sqlalchemy.orm.session.Session.flush

## Co to commit?

http://docs.sqlalchemy.org/en/latest/orm/session_api.html#sqlalchemy.orm.session.Session.commit

## Co to rollback?

http://docs.sqlalchemy.org/en/latest/orm/session_api.html#sqlalchemy.orm.session.Session.rollback

## Jak się bronić przed "popsutymi" transakcjami?

* `try ... except ...`
* `contextmanager`

# 9. SQLAlchemy i Flask

Flask ma wtyczkę usprawniającą działanie z SQLAlchemy: http://flask-sqlalchemy.pocoo.org

Ale nie będziemy jej dziś używać - nie chcę zaciemniać obrazu.

# 10. Jak to w życiu bywa